<a href="https://colab.research.google.com/github/kjcnim/Learned/blob/main/TF_LAB10_5_NN%2C_ReLu%2C_Xavier%2C_Dropout%2C_and_Adam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# 이번 LAB에서는 overfitting을 예방하는 방법인 Dropout 방법을 사용해본다.
# Dropout 방법이란 layer의 선을 끊어버리는 것이다.
# 그래서 모든 layer에 연결된 선을 다 학습시키지 않고 일부만 학습시켜서 overfitting을 예방하게 되는 것이다.
# Lab10-4에서 Dropout 기능만 추가하고 나머지 모든 코드는 동일하다.

import numpy as np
import random
import tensorflow as tf

#for reproductibility
random.seed(777) 
learning_rate = 0.001
batch_size = 100
training_epochs = 15
nb_classes = 10

# Dropout rate!!
# 학습할 때 30%만 가동한다는 것이다. 나머지 70%는 끊어버린다..
drop_rate = 0.3 

(x_train, y_train), (x_test2, y_test) = tf.keras.datasets.mnist.load_data()
print(x_train.shape)

#train과 test set으로 나눔
x_train = x_train.reshape(x_train.shape[0], 28 * 28)
x_test = x_test2.reshape(x_test2.shape[0], 28 * 28)

y_train = tf.keras.utils.to_categorical(y_train, nb_classes)
y_test = tf.keras.utils.to_categorical(y_test, nb_classes)

tf.model = tf.keras.Sequential()

# 첫번째 layer 설정
# 초기화 값을 설정한다. Tensor Flow_2 에서는 glorot_normal이 Xavier를 나타낸다.
# 출력을 더 많이 시킨다. 이전 lab에서는 256이었는데 2배로 들린 512를 사용한다.
tf.model.add(tf.keras.layers.Dense(input_dim = 784, units = 512, kernel_initializer='glorot_normal',activation = 'relu'))

# 모든 layer마다 Dropout을 적용한다.
tf.model.add(tf.keras.layers.Dropout(drop_rate))

# layer 설정
tf.model.add(tf.keras.layers.Dense(units = 512, kernel_initializer='glorot_normal', activation = 'relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))
tf.model.add(tf.keras.layers.Dense(units = 512, kernel_initializer='glorot_normal', activation = 'relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))
tf.model.add(tf.keras.layers.Dense(units = 512, kernel_initializer='glorot_normal', activation = 'relu'))
tf.model.add(tf.keras.layers.Dropout(drop_rate))

#결과 출력 layer에서는 activation function으로 softmax 사용
tf.model.add(tf.keras.layers.Dense(units = nb_classes, kernel_initializer='glorot_normal', activation = 'softmax'))

#이전엔 Gradientdescent를 사용했는데, optimizer로 Adam을 사용하면 더 좋다.
tf.model.compile(loss = 'categorical_crossentropy', optimizer = tf.keras.optimizers.Adam(lr = learning_rate), metrics='accuracy')
tf.model.summary()

tf.model.fit(x_train, y_train, batch_size = batch_size, epochs = training_epochs)


# predict 10 random hand-writing data
y_predicted = tf.model.predict(x_test)

#random하게 mnist의 data를 읽어와서 실제값과 예측값이 같은지를 보는 for문이다.
for x in range(0, 10):
    random_index = random.randint(0, x_test.shape[0]-1)
    print("index: ", random_index,
          "actual y: ", np.argmax(y_test[random_index]),
          "predicted y: ", np.argmax(y_predicted[random_index]))

# evaluate test set
evaluation = tf.model.evaluate(x_test, y_test)
print('loss: ', evaluation[0])
print('accuracy', evaluation[1])

# 원래 test할 때는, 모든 선을 사용해야하므로 rate를 1로 해야한다.
# 하지만 Tensor Flow 2에서는 고려 안해줘도 괜찮다.
# 출력 결과 LAB10 중에서 가장 높은 Accuracy를 보였다.
# 정리하자면, 기존의 MNIST 알고리즘에서(LAB10-1)
# Backpropagation's problem of sigmoid를 해결하기 위해 Activation function을 relu로 바꾸고(LAB10-2)
# 더 좋은 초기값 설정을 위해 Xavier를 이용하며(LAB10-3)
# 마지막으로 더 많은 학습을 위해 많은 layer와 출력을 사용할 때(LAB10-4)
# data만 그대로 기억하게 되어 발생하는 overfitting이 발생하는 것을 방지하기 위해 Dropout을 사용한다.(LAB10-5)
# 이 과정을 통해 accuracy를 0.9322에서 0.9735로 성능을 향상시켰다.

(60000, 28, 28)
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_26 (Dense)             (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 5